# Project I: Python Implementation For Algorithm 8 and 9 Shown Below.

## Group 3 Project Members:

### $\bullet$ Migabo Kisimbanyi Evariste
### $\bullet$ Lilly Kandagor Aengwo
### $\bullet$ Nnaa-Piam Alhassan
### $\bullet$ Janvier KWIZERA

Algorithm 8: Incremental Algorithm for computation of homology

Input : an increasing sequence of simplicial complexes $K_1 \subset K_2 \subset \dots K_n \subset K$ $\\$
Output: the Betti numbers $\beta_0(K), \beta_1(K),\dots, \beta_d (K)$ $\\$
$\beta_0 \leftarrow 0,\dots, \beta_d \leftarrow 0$; $\\$

for $i \leftarrow 1$ to $n$ do $\\$
    $\hspace{3mm}$ $d$ = dim($\sigma^i$); $\\$
     $\hspace{6mm}$ if $\sigma^i$ is positive then $\\$
    $\hspace{9mm}$ $\beta_d (K^i) \leftarrow  \beta_d (K^i) + 1$ $\\$
    $\hspace{6mm}$ end $\\$
    $\hspace{6mm}$ else $\\$
        $\hspace{9mm}$ if $d > 0$ then $\\$
            $\hspace{12mm}$ $\beta_{d-1} (K^i) \leftarrow  \beta_{d-1} (K^{i-1}) - 1$ $\\$
        $\hspace{9mm}$ end $\\$
    $\hspace{6mm}$ end $\\$
end $\\$


Algorithm 9: Gauss reduction of the boundary matrix
Input : a boundary matrix $\Delta$ $\\$
Output: a reduced matrix $\hat{\Delta}$ $\\$
for $j \leftarrow 1$ to $n$ do $\\$
    $\hspace{3mm}$ while there exists $i < j$ with $\delta(i) = \delta(j)$ do $\\$
    $\hspace{6mm}$ add column $i$ to column $j$ $\\$
    end $\\$
end


Implement the Incremental Algorithm (8) and Gauss Reduction
Algorithm (9). Incremental Algorithm. The input should be a simplicial complex $K$ (as a list or dictionary) or a filtra-
tion of subcomplexes $K_1 \subset K_2 \subset \dots K_n \subset K$. $\\$

The boundary matrix is defined as : 
$$  \Delta_{i,j} = \begin{cases} 
1,\hspace{1mm}\text{if}\hspace{1mm}\sigma^i \hspace{1mm}\text{is a face of}\hspace{1mm}\sigma^j\hspace{1mm}\text{and}\hspace{1mm} |\sigma^i| = |\sigma^j| -1 \\ 
0, \hspace{1mm}\text{otherwise}.
\end{cases} $$

The process of reducing columns to zero is called Gauss reduction. $\\$
For any $j$ such that $1 \leq j \leq n$, define $\\$

$$\delta(j) = \max\{i : \Delta_{i,j} \neq 0\}$$
If $\Delta_{i,j} = 0$ for all $i$, we way that $\delta(j)$ is undefined. We say that the boundary matrix $\Delta$ is reduced if the map $\delta$ is injective on its domain.

In [1]:
import numpy as np

def build_boundary_matrix(simplices):
    n = len(simplices)
    boundary_matrix = np.zeros((n, n), dtype=int)
    simplex_index = {tuple(simplices[i]): i for i in range(n)}
    
    for j in range(n):
        simplex = simplices[j]
        if len(simplex) > 1:
            for i in range(len(simplex)):
                face = tuple(simplex[:i] + simplex[i+1:])
                if face in simplex_index:
                    boundary_matrix[simplex_index[face], j] = 1
    
    return boundary_matrix
    
def reduce_boundary_matrix(D):
    def phi(column):
        nonzero_indices = np.where(column != 0)[0]
        return nonzero_indices[-1] if nonzero_indices.size > 0 else None

    n = D.shape[1]
    for j in range(n):
        while True:
            phi_j = phi(D[:, j])
            if phi_j is None:
                break

            found = False
            for i in range(j):
                if phi(D[:, i]) == phi_j:
                    D[:, j] = (D[:, j] + D[:, i]) % 2
                    found = True
                    break

            if not found:
                break

    return D

def ispositive(matrix):
    n = matrix.shape[1]
    sigmas = np.ones(n, dtype=int)
    for j in range(n):
        if np.any(matrix[:, j] == 1):
            sigmas[j] = 0
    return sigmas

def calculate_betti_numbers(simplices):
    boundary_matrix = build_boundary_matrix(simplices)
    reduced_matrix = reduce_boundary_matrix(boundary_matrix)
    positivity_vector = ispositive(reduced_matrix)
    
    max_dimension = max(len(simplex) - 1 for simplex in simplices)
    betti_numbers = [0] * (max_dimension + 1)
    
    for i in range(len(simplices)):
        d = len(simplices[i]) - 1
        if positivity_vector[i] == 1:
            betti_numbers[d] += 1
        elif d > 0:
            betti_numbers[d - 1] -= 1
    
    return betti_numbers

# Example usage
simplices_1 = [
    (0,),        # 0-simplex
    (1,),        # 0-simplex
    (2,),        # 0-simplex
    (0, 1),      # 1-simplex
    (1, 2),      # 1-simplex
    (0, 2)       # 1-simplex
]

#print("Boundary Matrix in case of a circle:")
#print(build_boundary_matrix(simplices_1))
#print("")

#("Reduced Matrix in case of a circle:")
#print(reduce_boundary_matrix(build_boundary_matrix(simplices_1)))
#print("")

print("Betti Numbers of a circle:")
print(calculate_betti_numbers(simplices_1))
print("")


# Example usage
simplices_2 = [
    (0,),        # 0-simplex
    (1,),        # 0-simplex
    (2,),        # 0-simplex
    (3,),        # 0-simplex
    (0, 1),      # 1-simplex
    (0, 2),      # 1-simplex
    (0, 3),      # 1-simplex
    (1, 2),      # 1-simplex
    (1, 3),      # 1-simplex
    (2, 3),      # 1-simplex
    (0,1,2),     # 2-simplex
    (1,2,3),     # 2-simplex
    (0,2,3),     # 2-simplex
    (0,1,3)      # 2-simplex
]

#print("Boundary Matrix in case of a sphere:")
#print(build_boundary_matrix(simplices_2))
#print("")

#print("Reduced Matrix in case of a sphere:")
#print(reduce_boundary_matrix(build_boundary_matrix(simplices_2)))
#print("")

print("Betti Numbers of a sphere:")
print(calculate_betti_numbers(simplices_2))
print("")



torus = [[1], [2], [3], [4], [5], [6], [7], [8], [9],
             [1, 2], [2, 3], [1, 3],
             [5, 9], [8, 9], [5, 8],
             [4, 6], [6, 7], [4, 7],
             [1, 5], [4, 5], [1, 4],
             [2, 9], [6, 9], [2, 6],
             [3, 8], [7, 8], [3, 7],
             [1, 9], [3, 9], [1, 8],
             [4, 9], [6, 8], [5, 7],
             [1, 6], [2, 7], [1, 7],
             [1, 5, 9], [1, 2, 9], [2, 3, 9], [3, 8, 9], [1, 3, 8], [1, 5, 8],
             [4, 5, 9], [4, 6, 9], [6, 8, 9], [6, 7, 8], [5, 7, 8], [4, 5, 7],
             [1, 4, 6], [1, 2, 6], [2, 6, 7], [2, 3, 7], [1, 3, 7], [1, 4, 7]]
#print("Boundary Matrix in case of a torus:")
#print(build_boundary_matrix(torus))
#print("")

#print("Reduced Matrix torus:")
#print(reduce_boundary_matrix(build_boundary_matrix(torus)))
#print("")

print("Betti Numbers of a torus:")
print(calculate_betti_numbers(torus))

Betti Numbers of a circle:
[1, 1]

Betti Numbers of a sphere:
[1, 0, 1]

Betti Numbers of a torus:
[1, 2, 1]
